In [ ]:
import os

os.environ["PRIVBAYES_BIN"] = "./ydnpd/synthesis/privbayes"

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import display, Markdown
import wandb

import ydnpd

In [ ]:
WANDB_GROUPS = ["test"]  # to add patectgan: "test2"

## Tasks

In [ ]:
total_task_size = 0
for idx, task in enumerate(ydnpd.span_utility_tasks(), start=1):
    print(f"{idx:2} {task}")
    total_task_size += task.size()
print(f"\nTotal task size: {total_task_size}")

## Load Results from W&B

In [ ]:
api = wandb.Api()

runs = api.runs(path=f"shlomihod/ydnpd")

utility_tasks_results = []

for run in runs:
    if run.group in WANDB_GROUPS:
        for step_data in run.history().to_dict(orient="records"):

            # TODO: why does this happen?
            if step_data["_step"] >= ydnpd.config.NUM_RUNS:
                print(f"Run {run.id} has more than {ydnpd.config.NUM_RUNS} steps")
                continue

            # TODO: why does this happen?
            for metric in step_data:
                if not metric.startswith("_") and step_data[metric] in ["NaN"]:
                    step_data[metric] = np.nan

            utility_tasks_results.append(dict(run.config) | {"evaluation": step_data})

## Utility-Related Tasks

In [ ]:
for experiments_name, experiments in ydnpd.config.ALL_EXPERIMENTS.items():

    display(Markdown(f"## {experiments_name}"))

    for metric in ydnpd.EVALUATION_METRICS:
        if metric not in utility_tasks_results[0]["evaluation"]:
            continue

        display(Markdown(f"### {metric}"))

        gs = ydnpd.UtilityTask.plot(
            utility_tasks_results, experiments, metric=metric
        )

        for g in gs:
            display(g.fig)
            plt.close(g.fig)

    for metric in ydnpd.EVALUATION_METRICS:
        if metric not in utility_tasks_results[0]["evaluation"]:
            continue

        display(Markdown(f"## {metric}"))
        with pd.option_context("display.max_rows", None):
            display(
                ydnpd.UtilityTask.evaluate(
                    utility_tasks_results, experiments, metric
                )
                .multiply(100)
                .round(1)
            )